In [1]:
# all import statements needed for the project, for example:
import math
import bs4
import pandas as pd
import pandas
import requests 
import sqlalchemy as db
from pandas import read_parquet
from pyarrow.parquet import ParquetDataset
from bs4 import BeautifulSoup
import re
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
from numba import cuda,jit
from datetime import datetime
import datetime
import glob
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session
import matplotlib.pyplot as plt
import sqlite3
from datetime import datetime
import matplotlib as mpl
from scipy import stats
import numpy as np
from keplergl import KeplerGl
from pandas import DataFrame

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
redius = 6378.137
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"
TAXI_DIR = 'taxi_files'
QUERY_DIRECTORY = "queries"
WEATHER_DIR = "weather_files"

zones_df =gpd.read_file(r"taxi_zones.shp")
zones_df.to_csv("taxi_zones.csv")
zones_df = zones_df.to_crs(4326)
zones_df["lat"] = zones_df.centroid.x
zones_df["lon"] = zones_df.centroid.y
zones_data = []

zones_pickupID = zones_df.drop(
        ["OBJECTID","Shape_Leng",
         "Shape_Area","zone",
         "borough","geometry"],
        axis="columns").rename(
        columns={
        "LocationID":"PULocationID",
        "lon":"pickup_latitude",
        "lat":"pickup_longitude"
        })
zones_dropoffID = zones_df.drop([
                "OBJECTID","Shape_Leng",
                "Shape_Area","zone",
                "borough","geometry"],
                axis="columns").rename(
                columns={
                "LocationID":"DOLocationID",
                "lon":"dropoff_latitude",
                "lat":"dropoff_longitude"
                })

/var/folders/4c/k607jfy56yv9kqkzvbvdnbcr0000gn/T/ipykernel_5571/269685051.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zones_df["lat"] = zones_df.centroid.x
/var/folders/4c/k607jfy56yv9kqkzvbvdnbcr0000gn/T/ipykernel_5571/269685051.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zones_df["lon"] = zones_df.centroid.y


In [4]:

# Calculate the distance through coordinates
def calculate_distance(from_coord, to_coord):
    lat1, lon1 = from_coord
    lat2, lon2 = to_coord

    diff_lat = math.radians(lat2 - lat1)
    diff_lon = math.radians(lon2 - lon1)
    
    cal_dis = (math.sin(diff_lat / 2) * math.sin(diff_lat / 2) +
             math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
             math.sin(diff_lon / 2) * math.sin(diff_lon / 2))
    cal_dis = 2 * math.atan2(math.sqrt(cal_dis), math.sqrt(1 - cal_dis))
    distance = redius * cal_dis

    return distance
    

# Add the coordinate columns to the dataframe
def add_distance_column(dataframe):
    dataframe["trip_distance"] = dataframe.apply(
                                lambda row: calculate_distance((row.pickup_longitude, row.pickup_latitude), 
                                                               (row.dropoff_longitude, row.dropoff_latitude)), 
                                                                axis=1)
    return dataframe

In [5]:
### Part 1: Data Preprocessing

In [6]:
# Get the download links of yellow taxi from 2009-01 to 2015-06
def find_taxi_csv_urls():
    response = requests.get(TAXI_URL)
    soup = BeautifulSoup(response.text, "html.parser")
    url_tags = soup.find_all("a", title="Yellow Taxi Trip Records")
    all_csv_urls = []
    pattern = "2009|2010|2011|2012|2013|2014|2015-0[1-6]"

    for url_tag in url_tags:
        if re.findall(pattern, url_tag["href"]):
            all_csv_urls.append(url_tag["href"])

    return all_csv_urls

In [7]:
def load_data_from_filename(filename):
    data = pd.read_parquet(filename)
    return data

In [38]:
# clean monthly yellow taxi data
def get_and_clean_month_taxi_data(url:str):
    file_name = url.split('/')[-1]
    if not os.path.exists(f'{TAXI_DIR}/{file_name}'):
        with open(f'{TAXI_DIR}/{file_name}', 'wb') as f:
            res = requests.get(url)
            f.write(res.content)

    dataframe = load_data_from_filename(f'{TAXI_DIR}/{file_name}')
    # Delete the useless columns
    # Attention: year 2009 and 2010 have unexpected columns' names.
    if "2010-" in file_name:
        dataframe = dataframe[
                            (dataframe["passenger_count"]> 0)
                            & (dataframe["total_amount"] > 0)
                                ]
        dataframe.drop([
                        "vendor_id", "dropoff_datetime", "rate_code", "store_and_fwd_flag",
                        "payment_type", "fare_amount", "surcharge",
                        "mta_tax","tolls_amount","passenger_count", 
                        "total_amount"
                        ], axis=1, inplace=True)
        
    elif "2009-" in file_name:
        dataframe = dataframe[
                            (dataframe["Passenger_Count"]> 0)
                            & (dataframe["Total_Amt"] > 0)
                                ]
        
        dataframe.drop([
                        "vendor_name", "Trip_Dropoff_DateTime", "Rate_Code",
                        "store_and_forward", "Payment_Type", "Fare_Amt", 
                        "surcharge", "mta_tax","Tolls_Amt",
                        "Passenger_Count",  'Total_Amt'
                        ], axis=1, inplace=True)
        
    else:
        dataframe = dataframe[
                            (dataframe["passenger_count"]> 0)
                            & (dataframe["total_amount"] > 0)
                                ]
        
        
        dataframe.drop([
                        "RatecodeID", "tolls_amount", "payment_type", 
                        "store_and_fwd_flag","mta_tax", "improvement_surcharge", 
                        "fare_amount", "extra","congestion_surcharge",
                        "airport_fee","VendorID", "tpep_dropoff_datetime", 
                        "passenger_count", "total_amount"
                        ], axis=1, inplace=True)
    
    # Unify the columns' names of year 2009 and 2012
    if "2010-" in file_name or "2009-" in file_name:
         dataframe.columns = [
                            "pickup_datetime", "trip_distance", "pickup_longitude", 
                             "pickup_latitude", "dropoff_longitude", "dropoff_latitude", 
                             "tip_amount"
                            ]
            
    else:
        # Change the IDs to the pick-up and drop-off coordinates, adding columns
        df = pd.merge(dataframe, zones_pickupID, on = "PULocationID")
        dataframe = pd.merge(df, zones_dropoffID, on = "DOLocationID")
        dataframe.drop(["PULocationID", "DOLocationID"], axis=1, inplace=True)
        dataframe.columns = [
            'pickup_datetime', 'trip_distance', 'tip_amount',
            'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'
                            ]
    

    # Remove the invalide datas
    dataframe = dataframe[
        (dataframe["pickup_longitude"] >= -74.242330)
         & (dataframe["pickup_latitude"] <= 40.908524)
         & (dataframe["pickup_longitude"] <= -73.717047)
         & (dataframe["pickup_latitude"] >= 40.560445)
         & (dataframe["dropoff_longitude"] >= -74.24233)
         & (dataframe["dropoff_latitude"] <= 40.908524)
         & (dataframe["dropoff_longitude"] <= -73.717047)
         & (dataframe["dropoff_latitude"] >= 40.560445)
    ]
    
    # Randomly choose a sample from every month.
    # Accumulated sample size is approximately to the uber data amount
    dataframe = dataframe.sample(n=2550)
    
    return dataframe

In [39]:
# clean uber data
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df.drop(columns = ["key", "Unnamed: 0"])
    df = df[
            (df["pickup_longitude"] >= -74.242330)
            & (df["pickup_latitude"] <= 40.908524)
            & (df["pickup_longitude"] <= -73.717047)
            & (df["pickup_latitude"] >= 40.560445)
            & (df["dropoff_longitude"] >= -74.24233)
            & (df["dropoff_latitude"] <= 40.908524)
            & (df["dropoff_longitude"] <= -73.717047)
            & (df["dropoff_latitude"] >= 40.560445)
            & (df["passenger_count"] > 0)
            & (df["fare_amount"] > 0 )
            ]

    df.drop(["passenger_count","fare_amount"], axis=1, inplace=True)
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"]).apply(lambda t: t.replace(tzinfo=None))
    
    return df    

In [11]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    uber_dataframe= uber_dataframe[uber_dataframe["trip_distance"] > 0]
    return uber_dataframe

In [13]:
#clean weather data, and deal with the datetime
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    df2 = df.loc[:, ["DATE", "HourlyWindSpeed",'HourlyPrecipitation']]
    df2["DATE HOUR"] = df2["DATE"].apply(lambda x:x[:-6])
    df2.drop(columns = ["DATE"], axis=1, inplace=True)
    df2["HourlyPrecipitation"].replace("T", 0.001)
    
    df2["HourlyWindSpeed"] = df2["HourlyWindSpeed"].apply(pd.to_numeric, errors = "coerce")
    df2["HourlyPrecipitation"] = df2["HourlyPrecipitation"].apply(pd.to_numeric, errors = "coerce")
    
    df2["HourlyWindSpeed"].fillna(0, inplace=True)
    df2["HourlyPrecipitation"].fillna(0, inplace=True)
    
    df3= df2.groupby(["DATE HOUR"], as_index=False)["HourlyWindSpeed"].mean()
    
    df4= df2.groupby(["DATE HOUR"], as_index=False)["HourlyPrecipitation"].sum()
    
    df4= pd.merge(df3, df4, on="DATE HOUR", how="inner")
    
    df4["DATE HOUR"] = list(map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H").strftime("%Y-%m-%d %H:"), df4["DATE HOUR"]))
    
    return df4

In [40]:
#clean weather data, and deal with the datetime
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file)
    df2 = df.loc[:, ["DATE", "HourlyWindSpeed",'HourlyPrecipitation']]
    df2["DATE DAYLY"] = df2["DATE"].apply(lambda x:x[:-9])
    df2.drop(columns=["DATE"],axis= 1,inplace=True)
    df2["HourlyPrecipitation"].replace('T', 0.001)
    
    df2["HourlyWindSpeed"] = df2["HourlyWindSpeed"].apply(pd.to_numeric, errors = "coerce")
    df2["HourlyPrecipitation"] = df2["HourlyPrecipitation"].apply(pd.to_numeric, errors = "coerce")
    
    
    df2["HourlyWindSpeed"].fillna(0, inplace=True)
    df2["HourlyPrecipitation"].fillna(0, inplace=True)
        
    df3= df2.groupby(["DATE DAYLY"], as_index=False)["HourlyWindSpeed"].mean()
    
    df4= df2.groupby(["DATE DAYLY"], as_index=False)["HourlyPrecipitation"].sum()
    
    df4= pd.merge(df3, df4, on="DATE DAYLY", how='inner')
    df4["DatelyWindSpeed"] = df4["HourlyWindSpeed"]
    df4["DatelyPrecipitation"] = df4["HourlyPrecipitation"]
    
    
    df4.drop(columns=["HourlyPrecipitation","HourlyWindSpeed"],axis= 1,inplace=True)
    
    return df4

In [15]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []

    weather_csv_files = [
                       "2009_weather.csv","2010_weather.csv","2011_weather.csv",
                       "2012_weather.csv","2013_weather.csv","2014_weather.csv",
                       "2015_weather.csv"
                        ]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    daily_data.drop(daily_data.loc[daily_data['DATE DAYLY'] > "2015-06-31"].index, inplace=True)
    hourly_data.drop(hourly_data.loc[hourly_data['DATE HOUR'] > "2015-06-31"].index, inplace=True)
    
    hourly_data.columns = ["datetime", "wind_speed", "precipitation"]
    daily_data.columns = ["datetime", "wind_speed", "precipitation"]
    return hourly_data, daily_data

## Part 2 : Storing Data

In [18]:
engine = db.create_engine(DATABASE_URL, echo=True)

In [19]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA =  """
        CREATE TABLE IF NOT EXISTS hourly_weather_data
        (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            datetime DATETIME,
            precipitation FLOAT,
            wind_speed FLOAT
        )
        """

DAILY_WEATHER_SCHEMA = """
        CREATE TABLE IF NOT EXISTS daily_weather_data
        (
           id INTEGER PRIMARY KEY AUTOINCREMENT,
           datetime DATETIME,
           precipitation FLOAT,
           wind_speed FLOAT
        )
        """

TAXI_TRIPS_SCHEMA = """
        CREATE TABLE IF NOT EXISTS taxi_data
        (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pickup_datetime DATETIME,
            trip_distance FLOAT,
            tip_amount FLOAT,
            pickup_longitude FLOAT,
            pickup_latitude FLOAT,
            dropoff_longitude FLOAT,
            dropoff_latitude FLOAT 
        )
        """

UBER_TRIPS_SCHEMA = """
        CREATE TABLE IF NOT EXISTS uber_data
        (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pickup_datetime DATETIME,
            trip_distance FLOAT,
            fare_amount FLOAT,
            pickup_longitude FLOAT,
            pickup_latitude FLOAT,
            dropoff_longitude FLOAT,
            dropoff_latitude FLOAT
        )
        """

In [20]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [21]:
# create the tables with the schema files
with engine.connect() as connection:
    taxi_trips = connection.execute(TAXI_TRIPS_SCHEMA)
    uber_trips = connection.execute(TAXI_TRIPS_SCHEMA)
    daily_weather = connection.execute(DAILY_WEATHER_SCHEMA)
    hourly_weather = connection.execute(HOURLY_WEATHER_SCHEMA)

2022-12-10 20:37:10,274 INFO sqlalchemy.engine.Engine 
        CREATE TABLE IF NOT EXISTS taxi_data
        (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pickup_datetime DATETIME,
            trip_distance FLOAT,
            tip_amount FLOAT,
            pickup_longitude FLOAT,
            pickup_latitude FLOAT,
            dropoff_longitude FLOAT,
            dropoff_latitude FLOAT 
        )
        
2022-12-10 20:37:10,275 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-10 20:37:10,278 INFO sqlalchemy.engine.Engine COMMIT
2022-12-10 20:37:10,279 INFO sqlalchemy.engine.Engine 
        CREATE TABLE IF NOT EXISTS taxi_data
        (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pickup_datetime DATETIME,
            trip_distance FLOAT,
            tip_amount FLOAT,
            pickup_longitude FLOAT,
            pickup_latitude FLOAT,
            dropoff_longitude FLOAT,
            dropoff_latitude FLOAT 
        )
        
2022-12-10 20:37:10,279 

In [36]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    df_list = []
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        dataframe = dataframe[dataframe["trip_distance"]> 0]
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        _df = dataframe
        _df.to_sql("taxi_trips", con=engine, index=False, if_exists="append")
        df_list.append(_df)
    
    
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    taxi_data.to_csv(f'finally_taxi_data.csv', index=False, header=taxi_data.columns, encoding="utf-8")

In [37]:
def add_uber_data_to_db():
    uber_data = get_uber_data()
    uber_data.to_csv(f'finally_uber_data.csv', index=False, header=uber_data.columns, encoding='utf-8')
    
    for i in range(40):
        uber_df = uber_data[i * 5000:(i + 1)*5000]
        uber_df.to_sql('uber_trips', con=engine, index=False, if_exists='append')

In [ ]:
def add_weather_data_to_db():
    hourly_data, daily_data = load_and_clean_weather_data()

    daily_data.to_sql("daily_weather", con=engine, index=False, if_exists="append")
    daily_data.to_csv(f'finally_daily_weather_data.csv', index=False, header=daily_data.columns, encoding="utf-8")

    # Import the hourly weather data to the table in 13 batches and
    # every time has amount of around 5000
    hourly_data.to_csv(f'finally_hourly_weather_data.csv', index=False, header=hourly_data.columns, encoding="utf-8")
    for i in range(13):
        hourly_df = hourly_data[i * 5000:(i + 1)*5000]
        hourly_df.to_sql("hourly_weather", con=engine, index=False, if_exists='append')

### Add data to DataBase

In [23]:
# When we call these three functions, we could get add data to our database
# get_and_clean_taxi_data()
# add_weather_data_to_db()
# add_uber_data_to_db()

## Part 3 : Understanding Data

In [26]:
#Question 1: 
def get_hour_popular_yellow_taxi():
    with engine.connect() as connection:
        query_sql = f"select strftime('%H',pickup_datetime) AS cur_hour,count(*) as num from taxi_trips GROUP BY cur_hour order by num desc;"
        # Write the query to the query file
        with open(f'{QUERY_DIRECTORY}/question1.sql', "w") as f:
            f.write(query_sql)
        # Execute the query
        result = connection.execute(query_sql)
        return [r[0] for r in result]
    
print(get_hour_popular_yellow_taxi())

2022-12-10 20:44:38,551 INFO sqlalchemy.engine.Engine select strftime('%H',pickup_datetime) AS cur_hour,count(*) as num from taxi_trips GROUP BY cur_hour order by num desc;
2022-12-10 20:44:38,553 INFO sqlalchemy.engine.Engine [raw sql] ()
['19', '18', '20', '21', '22', '14', '17', '12', '13', '23', '15', '09', '11', '08', '10', '16', '00', '07', '01', '02', '06', '03', '04', '05']


In [27]:
# Question 2:
def get_weekday_popular_uber():
    weekdays_dict = {
        '0': 'Sun',
        '1': 'Mon',
        '2': 'Tue',
        '3': 'Wed',
        '4': 'Thur',
        '5': 'Fri',
        '6': 'Sat'
    }
    with engine.connect() as connection:
        query_sql = f"select strftime('%w',pickup_datetime) AS cur_weekday,count(*) as num from uber_trips GROUP BY cur_weekday order by num desc;"
        # Write the query to the query file
        with open(f'{QUERY_DIRECTORY}/question2.sql', "w") as f:
            f.write(query_sql)
        # Execute the query
        result = connection.execute(query_sql)
        return [weekdays_dict[r[0]] for r in result]
    
print(get_weekday_popular_uber())

2022-12-10 20:44:41,105 INFO sqlalchemy.engine.Engine select strftime('%w',pickup_datetime) AS cur_weekday,count(*) as num from uber_trips GROUP BY cur_weekday order by num desc;
2022-12-10 20:44:41,106 INFO sqlalchemy.engine.Engine [raw sql] ()
['Fri', 'Sat', 'Thur', 'Wed', 'Tue', 'Sun', 'Mon']


In [30]:
# Question 3
def get_sum_trip_distence():
    with engine.connect() as connection:
        distence_query_sql = """
                            select sum(trip_distance)*0.95 as total_distance
                            from 
                            (select * from taxi_trips where strftime('%Y-%m', pickup_datetime) = '2013-07'
                             union all
                             select * from taxi_trips where strftime('%Y-%m', pickup_datetime) = '2013-07')

                            """
        # Write the query to the query file
        with open(f'{QUERY_DIRECTORY}/question3.sql', "w") as f:
            f.write(distence_query_sql)
         # Execute the query
        result = list(connection.execute(distence_query_sql))
        distence = 0
        for row in result:
            print(row)
    
get_sum_trip_distence()

2022-12-10 21:02:18,944 INFO sqlalchemy.engine.Engine 
                            select sum(trip_distance)*0.95 as total_distance
                            from 
                            (select * from taxi_trips where strftime('%Y-%m', pickup_datetime) = '2013-07'
                             union all
                             select * from taxi_trips where strftime('%Y-%m', pickup_datetime) = '2013-07')

                            
2022-12-10 21:02:18,946 INFO sqlalchemy.engine.Engine [raw sql] ()
(14069.518999999997,)


In [33]:
# Question4
def get_2019_trips_info():
    with engine.connect() as connection:
        q4 = """
        select strftime('%Y-%m-%d', pickup_datetime) as date, count(*) as trips_amount, sum(trip_distance) as distance
        from
        (select * from taxi_trips where strftime('%Y', pickup_datetime) = '2014'
        union all
        select * from uber_trips where strftime('%Y', pickup_datetime) = '2014')
        group by strftime('%m-%d', pickup_datetime)
        order by trips_amount desc
        limit 10
        """
        
        # create the query.sql file
        with open(f'{QUERY_DIRECTORY}/question4.sql', "w") as f:
            f.write(q4)
            
        # check the result
        result = list(connection.execute(q4))
        for row in result:
            print(row)
            
get_2019_trips_info()

2022-12-10 21:04:27,985 INFO sqlalchemy.engine.Engine 
        select strftime('%Y-%m-%d', pickup_datetime) as date, count(*) as trips_amount, sum(trip_distance) as distance
        from
        (select * from taxi_trips where strftime('%Y', pickup_datetime) = '2014'
        union all
        select * from uber_trips where strftime('%Y', pickup_datetime) = '2014')
        group by strftime('%m-%d', pickup_datetime)
        order by trips_amount desc
        limit 10
        
2022-12-10 21:04:27,987 INFO sqlalchemy.engine.Engine [raw sql] ()
('2014-04-12', 231, 734.2488284338865)
('2014-03-29', 228, 660.2575145232145)
('2014-02-01', 222, 678.4233201164744)
('2014-02-28', 221, 631.1204102700872)
('2014-02-07', 219, 652.8391061550848)
('2014-05-16', 218, 639.2120721481293)
('2014-11-01', 214, 666.2615178182924)
('2014-03-01', 213, 596.5033025222677)
('2014-12-11', 211, 622.6627065275238)
('2014-12-13', 207, 649.18667000848)


In [35]:
# Question 5
def get_windest_days():
    with engine.connect() as connection:
        q5 = """
            select strftime('%Y-%m-%d', pickup_datetime) as date, wind_speed, count(*) as trips
            from 
            (select * from taxi_trips where strftime('%Y', pickup_datetime) = '2014'
            union all
            select * from uber_trips where strftime('%Y', pickup_datetime) = '2014')
            left join daily_weather on strftime('%Y-%m-%d', pickup_datetime) = datetime
            group by strftime('%m-%d', pickup_datetime)
            order by wind_speed desc
            limit 10
            """
         # create the query file
        with open(f'{QUERY_DIRECTORY}/question5.sql', "w") as f:
                f.write(q5)
        # check the result
        result = list(connection.execute(q5))

        for row in result:
            print(row)
        
get_windest_days()

2022-12-10 21:04:55,514 INFO sqlalchemy.engine.Engine 
            select strftime('%Y-%m-%d', pickup_datetime) as date, wind_speed, count(*) as trips
            from 
            (select * from taxi_trips where strftime('%Y', pickup_datetime) = '2014'
            union all
            select * from uber_trips where strftime('%Y', pickup_datetime) = '2014')
            left join daily_weather on strftime('%Y-%m-%d', pickup_datetime) = datetime
            group by strftime('%m-%d', pickup_datetime)
            order by wind_speed desc
            limit 10
            
2022-12-10 21:04:55,515 INFO sqlalchemy.engine.Engine [raw sql] ()
('2014-03-13', 12.923076923076923, 182)
('2014-01-07', 12.56, 163)
('2014-01-02', 12.352941176470589, 133)
('2014-02-13', 11.781818181818181, 132)
('2014-03-29', 11.666666666666666, 228)
('2014-03-26', 10.52, 176)
('2014-01-03', 9.710526315789474, 105)
('2014-02-14', 9.421052631578947, 170)
('2014-03-30', 8.790697674418604, 148)
('2014-02-15', 8.575, 197)

In [41]:
# Question6
def get_hurricane_trips_info():
    with engine.connect() as connection:

        start_date = datetime.date(2012, 10, 22)
        date_dict = {}
        for i in range(16):
            for j in range(24):
                date_dict[f'{str(start_date)} {j:02d}'] = {
                    'trips_num': 0, 'precipitation': 0, 'wind_speed': 0
                }
                j += 1
            start_date = start_date + datetime.timedelta(days=1)
              
        query1 = """
            select strftime('%Y-%m-%d %H',pickup_datetime) as cur_hour,count(*) as
            trips_num FROM uber_trips where pickup_datetime>='2012-10-22' and pickup_datetime<'2012-11-07' GROUP BY cur_hour;
        """
        result = list(connection.execute(query1))
        for row in result:
            cur_hour = row[0]
            trips_num = row[1]
            temp_dict = date_dict[cur_hour]
            temp_dict['trips_num'] = temp_dict['trips_num'] + trips_num
            
        query2 = """
            select strftime('%Y-%m-%d %H',pickup_datetime) as cur_hour,count(*) as
            trips_num FROM taxi_trips where pickup_datetime>='2012-10-22' and pickup_datetime<'2012-11-07' GROUP BY cur_hour;
        """
        result = list(connection.execute(query2))
        for row in result:
            cur_hour = row[0]
            trips_num = row[1]
            temp_dict = date_dict[cur_hour]
            temp_dict['trips_num'] = temp_dict['trips_num'] + trips_num
            
            
        query3 = """
            select * from hourly_weather where datetime>='2012-10-22' AND datetime<'2012-10-23';
        """
        result = list(connection.execute(query3))
        for row in result:
            cur_hour = str(row[1]).replace(':', '')
            precipitation = row[2]
            wind_speed = row[3]
            temp_dict = date_dict[cur_hour]
            temp_dict['precipitation'] = precipitation
            temp_dict['wind_speed'] = wind_speed
        

        with open(f'{QUERY_DIRECTORY}/question6.sql', "w") as f:
            f.write(query1)
            f.write(query2)
            f.write(query3)
        
        return date_dict
    
get_hurricane_trips_info()

2022-12-10 21:18:09,965 INFO sqlalchemy.engine.Engine 
            select strftime('%Y-%m-%d %H',pickup_datetime) as cur_hour,count(*) as
            trips_num FROM uber_trips where pickup_datetime>='2012-10-22' and pickup_datetime<'2012-11-07' GROUP BY cur_hour;
        
2022-12-10 21:18:09,966 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-10 21:18:10,047 INFO sqlalchemy.engine.Engine 
            select strftime('%Y-%m-%d %H',pickup_datetime) as cur_hour,count(*) as
            trips_num FROM taxi_trips where pickup_datetime>='2012-10-22' and pickup_datetime<'2012-11-07' GROUP BY cur_hour;
        
2022-12-10 21:18:10,047 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-10 21:18:10,113 INFO sqlalchemy.engine.Engine 
            select * from hourly_weather where datetime>='2012-10-22' AND datetime<'2012-10-23';
        
2022-12-10 21:18:10,113 INFO sqlalchemy.engine.Engine [raw sql] ()


{'2012-10-22 00': {'trips_num': 3, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 01': {'trips_num': 0, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 02': {'trips_num': 3, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 03': {'trips_num': 1, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 04': {'trips_num': 0, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 05': {'trips_num': 1, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 06': {'trips_num': 6, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 07': {'trips_num': 5, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 08': {'trips_num': 8, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 09': {'trips_num': 10, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 10': {'trips_num': 8, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 11': {'trips_num': 14, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 12': {'trips_num': 12, 'precipitation': 0, 'wind_speed': 0},
 '2012-10-22 13': {'trips_num': 12, 'precipitati

## Extra Credit

In [42]:
SUNSET_SUNRISE_SCHEMA = """
    CREATE TABLE IF NOT EXISTS sunset_sunrise (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sunrise_time DATETIME,
        sunset_time DATETIME
    );
"""

# create a new table
with open(DATABASE_SCHEMA_FILE, "a") as f:
    f.write(SUNSET_SUNRISE_SCHEMA)

with engine.connect() as connection:
    connection.execute(SUNSET_SUNRISE_SCHEMA)

    
# clean the data
def load_and_clean_weather_sun_data(csv_file):
    df = pd.read_csv(csv_file, low_memory=False)
    df = df.loc[:, ["DATE", "Sunrise",'Sunset']]
    df = df[df['Sunrise'].notnull()]
    
    row_list = []
    for index, row in df.iterrows():
        record_date = str(row['DATE'])[0:10]
        sunrise = f"{int(row['Sunrise']):04d}"
        sunrise_time = f'{record_date} {sunrise[:2]}:{sunrise[2:]}:00'
        row['sunrise_time'] = sunrise_time
        sunset = f"{int(row['Sunset']):04d}"
        sunset_time = f'{record_date} {sunset[:2]}:{sunset[2:]}:00'
        row['sunset_time'] = sunset_time
        row_list.append(row)
    
    new_df = pd.DataFrame(row_list)
    new_df.drop(columns=["DATE", "Sunrise",'Sunset'], axis=1, inplace=True)
    return new_df


with engine.connect() as connection:
    test_sql = f"select * from sunset_sunrise;"
    result = list(connection.execute(test_sql)) 
    for row in result:
        print(row)

2022-12-10 21:21:52,426 INFO sqlalchemy.engine.Engine 
    CREATE TABLE IF NOT EXISTS sunset_sunrise (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sunrise_time DATETIME,
        sunset_time DATETIME
    );

2022-12-10 21:21:52,427 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-10 21:21:52,428 INFO sqlalchemy.engine.Engine COMMIT
2022-12-10 21:21:52,430 INFO sqlalchemy.engine.Engine select * from sunset_sunrise;
2022-12-10 21:21:52,430 INFO sqlalchemy.engine.Engine [raw sql] ()
(1, '2015-01-01 07:20:00', '2015-01-01 16:39:00')
(2, '2015-01-02 07:20:00', '2015-01-02 16:40:00')
(3, '2015-01-03 07:20:00', '2015-01-03 16:41:00')
(4, '2015-01-04 07:20:00', '2015-01-04 16:42:00')
(5, '2015-01-05 07:20:00', '2015-01-05 16:43:00')
(6, '2015-01-06 07:20:00', '2015-01-06 16:44:00')
(7, '2015-01-07 07:20:00', '2015-01-07 16:44:00')
(8, '2015-01-08 07:20:00', '2015-01-08 16:45:00')
(9, '2015-01-09 07:20:00', '2015-01-09 16:46:00')
(10, '2015-01-10 07:20:00', '2015-01-10 16:47:00')
(11

In [46]:
# Question we made: what is the total hired trips in 2015.01.31 after sunset
def get_sunset_trips_info():
     with engine.connect() as connection:
        total_trips = f"""
            select count(*) as total_trips from
            (select * from uber_trips where pickup_datetime<'2015-02-01' and
            pickup_datetime>=(select sunset_time from sunset_sunrise where sunset_time like '%2015-01-31%')
            union all
            select * from taxi_trips where pickup_datetime<'2015-02-01' and
            pickup_datetime>=(select sunset_time from sunset_sunrise where sunset_time like '%2015-01-31%'))
            """
        
        # write to qurey file
        with open(f'{QUERY_DIRECTORY}/extra_credit.sql', "w") as f:
            f.write(total_trips)
        
        result = list(connection.execute(total_trips))
        print(result)
        
get_sunset_trips_info()

2022-12-10 21:38:04,261 INFO sqlalchemy.engine.Engine 
            select count(*) as total_trips from
            (select * from uber_trips where pickup_datetime<'2015-02-01' and
            pickup_datetime>=(select sunset_time from sunset_sunrise where sunset_time like '%2015-01-31%')
            union all
            select * from taxi_trips where pickup_datetime<'2015-02-01' and
            pickup_datetime>=(select sunset_time from sunset_sunrise where sunset_time like '%2015-01-31%'))
            
2022-12-10 21:38:04,262 INFO sqlalchemy.engine.Engine [raw sql] ()
[(69,)]


## Part 4: Visualizing the Data

In [ ]:
#p4 q1 
#Create an appropriate visualization for the first query/question in part 3
def popular_hour():
    q1 = """
    select strftime('%H',pickup_datetime) AS cur_hour,count(*) as num
    from taxi_trips GROUP BY cur_hour
    """
    with engine.connect() as connection:

        test1 = connection.execute(q1)
        df1 = DataFrame(test1.fetchall())
        df1.columns = test1.keys()
    plt.bar(df1["cur_hour"], df1["num"], width = 0.2)
    plt.xlabel("hours")
    plt.ylabel("count")
    plt.title("count in each hour")
    plt.show()
popular_hour()

In [ ]:
#p4 q2
#Create a visualization that shows the average distance traveled per month
def get_avg_distance_per_month():
    
    query_sql = f"select strftime('%m',pickup_datetime) AS per_month,sum(trip_distance) as trip_distance from taxi_trips GROUP BY per_month;"
    
    with engine.connect() as connection:
        t4 = connection.execute(query_sql)
        df_4 = DataFrame(t4.fetchall())
        df_4.columns = t4.keys()
    mean = df_4.groupby(["per_month"], as_index=False)['trip_distance'].mean()
    k_lower =  df_4.groupby(["per_month"], as_index=False)['trip_distance'].quantile(0.05)
    k_upper = df_4.groupby(["per_month"], as_index=False)['trip_distance'].quantile(0.95) 
   
    plt.bar(df_4["per_month"],df_4["trip_distance"],yerr=[k_lower['trip_distance'].to_numpy(),k_upper['trip_distance'].to_numpy()], alpha=0.5,width = 0.2)
    plt.xlabel("per_month")
    plt.ylabel("trip distance")
    plt.title("the average distance traveled per month")
    plt.show()
get_avg_distance_per_month()